In [1]:
import pandas as pd
import numpy as np

# Manuscript data
# 1. expr -> expression matrix (36, 785)
# 2. meta -> metadata 
expr = pd.read_csv("ml_input/expression_log2_counts.tsv", sep="\t")
meta = pd.read_csv("ml_input/sample_metadata.tsv", sep="\t")

expr = expr.sort_values("sample_id").reset_index(drop=True)
meta = meta.sort_values("sample_id").reset_index(drop=True)

assert all(expr["sample_id"] == meta["sample_id"])



In [7]:
from sklearn.preprocessing import LabelEncoder

# Features 
X = expr.drop(columns=["sample_id"]).values
gene_names = expr.drop(columns=['sample_id']).columns.to_list()

# Labels: PD = 1 (positive), WD = 0 (negative)
y = (meta["nen_subtype"] == "PD").astype(int).values
# print(y)
# print(meta["nen_subtype"])

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=23
)



In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

lr = LogisticRegression(
    penalty="l2",
    solver="liblinear",
    class_weight="balanced",
    random_state=23
)

rf = RandomForestClassifier(
    n_estimators=1000,
    max_features="sqrt",
    class_weight="balanced",
    random_state=23
)


In [47]:
from sklearn.metrics import roc_auc_score

lr.fit(X_train, y_train)
rf.fit(X_train, y_train)

print("LR train AUC:", roc_auc_score(y_train, lr.predict_proba(X_train)[:, 1]))
print("LR test  AUC:", roc_auc_score(y_test,  lr.predict_proba(X_test)[:, 1]))

print("RF train AUC:", roc_auc_score(y_train, rf.predict_proba(X_train)[:, 1]))
print("RF test  AUC:", roc_auc_score(y_test,  rf.predict_proba(X_test)[:, 1]))


LR train AUC: 1.0
LR test  AUC: 0.9666666666666667
RF train AUC: 1.0
RF test  AUC: 1.0


In [48]:
for seed in [1, 7, 13, 23, 42, 101]:
    X_tr, X_te, y_tr, y_te = train_test_split(
        X, y, test_size=0.25, random_state=seed
    )
    lr.fit(X_tr, y_tr)
    auc = roc_auc_score(y_te, lr.predict_proba(X_te)[:,1])
    print(seed, auc)


1 1.0
7 1.0
13 1.0
23 1.0
42 1.0
101 1.0


In [51]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# ----------------------------
# Fixed data + labels
# ----------------------------
X_full = expr.drop(columns=["sample_id"]).values
y = (meta["nen_subtype"] == "PD").astype(int).values
gene_names = expr.drop(columns=["sample_id"]).columns.to_list()

# ----------------------------
# Gene-wise variance
# ----------------------------
gene_var = np.var(X_full, axis=0)
var_rank_idx = np.argsort(gene_var)[::-1]  # descending variance

# ----------------------------
# Logistic Regression config
# ----------------------------
lr = LogisticRegression(
    penalty="l2",
    solver="liblinear",
    class_weight="balanced",
    max_iter=1000
)

print("=== Logistic Regression ===")
print("penalty=l2 | solver=liblinear | class_weight=balanced")
print("Encoding: PD = 1, WD = 0")
print("Stratify: False")
print("-------------------------------------")

# ----------------------------
# Variance filter sweep
# ----------------------------
for n_genes in [10, 25, 30, 50, 100, 200, 300]:
    print(f"\n=== Top {n_genes} most variable genes ===")

    X = X_full[:, var_rank_idx[:n_genes]]

    for test_size in [0.25, 0.30]:
        aucs = []

        for seed in range(1, 151):
            X_tr, X_te, y_tr, y_te = train_test_split(
                X,
                y,
                test_size=test_size,
                random_state=seed
            )

            # Skip degenerate test splits
            if len(np.unique(y_te)) < 2:
                continue

            lr.fit(X_tr, y_tr)
            y_score = lr.predict_proba(X_te)[:, 1]
            aucs.append(roc_auc_score(y_te, y_score))

        if len(aucs) == 0:
            print(f"  Split {int((1-test_size)*100)}/{int(test_size*100)}: no valid splits")
        else:
            print(
                f"  Split {int((1-test_size)*100)}/{int(test_size*100)} | "
                f"N={len(aucs):3d} | "
                f"min={np.min(aucs):.3f} | "
                f"mean={np.mean(aucs):.3f} | "
                f"max={np.max(aucs):.3f}"
            )


=== Logistic Regression ===
penalty=l2 | solver=liblinear | class_weight=balanced
Encoding: PD = 1, WD = 0
Stratify: False
-------------------------------------

=== Top 10 most variable genes ===
  Split 75/25 | N=149 | min=0.786 | mean=0.986 | max=1.000
  Split 70/30 | N=150 | min=0.875 | mean=0.987 | max=1.000

=== Top 25 most variable genes ===
  Split 75/25 | N=149 | min=0.889 | mean=0.992 | max=1.000
  Split 70/30 | N=150 | min=0.833 | mean=0.991 | max=1.000

=== Top 30 most variable genes ===
  Split 75/25 | N=149 | min=0.889 | mean=0.990 | max=1.000
  Split 70/30 | N=150 | min=0.833 | mean=0.989 | max=1.000

=== Top 50 most variable genes ===
  Split 75/25 | N=149 | min=0.900 | mean=0.996 | max=1.000
  Split 70/30 | N=150 | min=0.867 | mean=0.995 | max=1.000

=== Top 100 most variable genes ===
  Split 75/25 | N=149 | min=0.950 | mean=1.000 | max=1.000
  Split 70/30 | N=150 | min=0.900 | mean=0.999 | max=1.000

=== Top 200 most variable genes ===
  Split 75/25 | N=149 | min=1.0

In [52]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# ----------------------------
# Data
# ----------------------------
X_full = expr.drop(columns=["sample_id"]).values
gene_names = expr.drop(columns=["sample_id"]).columns.to_list()
y = (meta["nen_subtype"] == "PD").astype(int).values  # PD=1, WD=0

# ----------------------------
# Model configs (match your earlier setup style)
# ----------------------------
lr = LogisticRegression(
    penalty="l2",
    solver="liblinear",
    class_weight="balanced",
    max_iter=1000
)

def make_rf(seed: int):
    return RandomForestClassifier(
        n_estimators=1000,
        max_features="sqrt",
        class_weight="balanced",
        random_state=seed
    )

print("=== Supervised feature selection: Welch t-test (PD vs WD) ===")
print("Labels: PD=1, WD=0 | Stratify: False")
print("LR: penalty=l2 | solver=liblinear | class_weight=balanced | max_iter=1000")
print("RF: n_estimators=1000 | max_features=sqrt | class_weight=balanced | random_state=seed")
print("------------------------------------------------------------\n")

# ----------------------------
# Welch t-test ranking (no scipy dependency)
# Returns indices sorted by ascending p-value (most significant first)
# ----------------------------
def welch_t_rank(X: np.ndarray, y_bin: np.ndarray) -> np.ndarray:
    X1 = X[y_bin == 1]
    X0 = X[y_bin == 0]

    n1 = X1.shape[0]
    n0 = X0.shape[0]

    m1 = X1.mean(axis=0)
    m0 = X0.mean(axis=0)

    v1 = X1.var(axis=0, ddof=1)
    v0 = X0.var(axis=0, ddof=1)

    # Welch t-statistic
    denom = np.sqrt((v1 / n1) + (v0 / n0))
    denom = np.where(denom == 0, np.nan, denom)
    t = (m1 - m0) / denom

    # Approximate ranking by |t| if p-values can't be computed reliably without scipy
    # (still a supervised ranking; many older notebooks effectively did this)
    # Use nan_to_num so constant genes don't break sorting
    score = np.nan_to_num(np.abs(t), nan=0.0, posinf=0.0, neginf=0.0)

    # Sort descending by |t|
    return np.argsort(score)[::-1]

# ----------------------------
# Evaluation helper
# ----------------------------
def eval_once(X_tr, X_te, y_tr, y_te, seed: int):
    # LR
    lr.fit(X_tr, y_tr)
    lr_auc = roc_auc_score(y_te, lr.predict_proba(X_te)[:, 1])

    # RF
    rf = make_rf(seed)
    rf.fit(X_tr, y_tr)
    rf_auc = roc_auc_score(y_te, rf.predict_proba(X_te)[:, 1])

    return lr_auc, rf_auc

topNs = [10, 25, 50, 75, 100, 150, 200]
splits = [0.25, 0.30]
seeds = range(1, 151)

# ============================================================
# MODE 1: "LEAKY / Global DE" (most likely historical)
# ============================================================
global_rank_idx = welch_t_rank(X_full, y)

print("### MODE 1: LEAKY / Global DE ranking (computed once on ALL samples) ###")
for n_genes in topNs:
    feat_idx = global_rank_idx[:n_genes]
    X = X_full[:, feat_idx]

    for test_size in splits:
        lr_aucs, rf_aucs = [], []

        for seed in seeds:
            X_tr, X_te, y_tr, y_te = train_test_split(
                X, y, test_size=test_size, random_state=seed
            )
            if len(np.unique(y_te)) < 2:
                continue

            lr_auc, rf_auc = eval_once(X_tr, X_te, y_tr, y_te, seed)
            lr_aucs.append(lr_auc)
            rf_aucs.append(rf_auc)

        if len(lr_aucs) == 0:
            print(f"TopN={n_genes:3d} | split {int((1-test_size)*100)}/{int(test_size*100)}: no valid splits")
        else:
            print(
                f"TopN={n_genes:3d} | split {int((1-test_size)*100)}/{int(test_size*100)} | "
                f"N={len(lr_aucs):3d} | "
                f"LR min/mean/max={np.min(lr_aucs):.3f}/{np.mean(lr_aucs):.3f}/{np.max(lr_aucs):.3f} | "
                f"RF min/mean/max={np.min(rf_aucs):.3f}/{np.mean(rf_aucs):.3f}/{np.max(rf_aucs):.3f}"
            )

print("\n")

# ============================================================
# MODE 2: TRAIN-ONLY DE (modern correct; included for comparison)
# ============================================================
print("### MODE 2: TRAIN-ONLY DE ranking (recomputed inside each split) ###")
for n_genes in topNs:
    for test_size in splits:
        lr_aucs, rf_aucs = [], []

        for seed in seeds:
            X_tr_full, X_te_full, y_tr, y_te = train_test_split(
                X_full, y, test_size=test_size, random_state=seed
            )
            if len(np.unique(y_te)) < 2:
                continue

            # DE ranking using TRAIN ONLY
            rank_idx = welch_t_rank(X_tr_full, y_tr)
            feat_idx = rank_idx[:n_genes]

            X_tr = X_tr_full[:, feat_idx]
            X_te = X_te_full[:, feat_idx]

            lr_auc, rf_auc = eval_once(X_tr, X_te, y_tr, y_te, seed)
            lr_aucs.append(lr_auc)
            rf_aucs.append(rf_auc)

        if len(lr_aucs) == 0:
            print(f"TopN={n_genes:3d} | split {int((1-test_size)*100)}/{int(test_size*100)}: no valid splits")
        else:
            print(
                f"TopN={n_genes:3d} | split {int((1-test_size)*100)}/{int(test_size*100)} | "
                f"N={len(lr_aucs):3d} | "
                f"LR min/mean/max={np.min(lr_aucs):.3f}/{np.mean(lr_aucs):.3f}/{np.max(lr_aucs):.3f} | "
                f"RF min/mean/max={np.min(rf_aucs):.3f}/{np.mean(rf_aucs):.3f}/{np.max(rf_aucs):.3f}"
            )


=== Supervised feature selection: Welch t-test (PD vs WD) ===
Labels: PD=1, WD=0 | Stratify: False
LR: penalty=l2 | solver=liblinear | class_weight=balanced | max_iter=1000
RF: n_estimators=1000 | max_features=sqrt | class_weight=balanced | random_state=seed
------------------------------------------------------------

### MODE 1: LEAKY / Global DE ranking (computed once on ALL samples) ###
TopN= 10 | split 75/25 | N=149 | LR min/mean/max=1.000/1.000/1.000 | RF min/mean/max=1.000/1.000/1.000
TopN= 10 | split 70/30 | N=150 | LR min/mean/max=1.000/1.000/1.000 | RF min/mean/max=1.000/1.000/1.000
TopN= 25 | split 75/25 | N=149 | LR min/mean/max=1.000/1.000/1.000 | RF min/mean/max=0.875/0.998/1.000
TopN= 25 | split 70/30 | N=150 | LR min/mean/max=1.000/1.000/1.000 | RF min/mean/max=0.929/0.999/1.000
TopN= 50 | split 75/25 | N=149 | LR min/mean/max=1.000/1.000/1.000 | RF min/mean/max=0.875/0.997/1.000
TopN= 50 | split 70/30 | N=150 | LR min/mean/max=1.000/1.000/1.000 | RF min/mean/max=0.929/